In [1]:
# Imports
import pandas as pd
import numpy as np
from numpy import array
import librosa
import librosa.display
import requests
import json
import logging
import os
from os import path
from io import BytesIO
import zipfile
import shutil
import glob
import math
import matplotlib.pyplot as plt

# Keras imports
import tensorflow as tf
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Convolution2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, merge, Input, BatchNormalization, LSTM, Reshape
from keras.layers.core import *
import keras_metrics

# Sci-kit learn imports for pre-processing
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
#!pip install keras_metrics
#!pip install librosa

The following is used to download any songs from the BeatSaber Community Site. This is similar to the download done in the beatmap data analysis, but it grabs only the information needed and also grabs the filename for song files that differ from the expected DifficultyName.dat filename scheme

In [3]:
def create_song_dict():
    song_dict = {}
    song_dict["time"] = []
    song_dict["type"] = []
    song_dict["value"] = []
    song_dict["line_index"] = []
    song_dict["line_layer"] = []
    song_dict["cut_direction"] = []
    song_dict["duration"] = []
    song_dict["width"] = []
    song_dict["object_type"] = []
    
    return song_dict

def add_event_to_dict(song_dict, time, event_type, value):
    """Appends the values for an event to the song's dictionary"""
    song_dict["time"].append(time)
    song_dict["type"].append(event_type)
    song_dict["value"].append(value)
    song_dict["line_index"].append(np.nan) # NA on events
    song_dict["line_layer"].append(np.nan) # NA on events
    song_dict["cut_direction"].append(np.nan) # NA on events
    song_dict["duration"].append(np.nan) # NA on events
    song_dict["width"].append(np.nan) # NA on events
    song_dict["object_type"].append("Event")

    
def add_note_to_dict(song_dict, time, line_index, line_layer, note_type, cut_direction):
    """Appends the values for a note (which includes bombs) to the song's dictionary"""
    song_dict["time"].append(time)
    song_dict["type"].append(note_type)
    song_dict["value"].append(np.nan) # NA on notes/bombs
    song_dict["line_index"].append(line_index)
    song_dict["line_layer"].append(line_layer)
    song_dict["cut_direction"].append(cut_direction)
    song_dict["duration"].append(np.nan) # NA on notes/bombs
    song_dict["width"].append(np.nan) # NA on notes/bombs
    
    if (note_type == 1) or (note_type == 0):
        song_dict["object_type"].append("Note")
    else:
        song_dict["object_type"].append("Bomb")

        
def add_obstacle_to_dict(song_dict, time, line_index, obstacle_type, duration, width):
    """Appends the values for an obstacle to the song's dictionary"""
    song_dict["time"].append(time)
    song_dict["type"].append(obstacle_type)
    song_dict["value"].append(np.nan)
    song_dict["line_index"].append(line_index) # NA on obstacles
    song_dict["line_layer"].append(np.nan) # NA on obstacles
    song_dict["cut_direction"].append(np.nan) # NA on obstacles
    song_dict["duration"].append(duration) # NA on obstacles
    song_dict["width"].append(width) # NA on obstacles
    song_dict["object_type"].append("Obstacle")

    
def get_pandas_song_map(row):
    """Get the pandas dataframe for a map file, or convert the song map file into a pandas representation"""
    
    orig_filename = "data/{}/{}".format(row["key"], row["map_filename"])
    new_filename = orig_filename.replace(".dat", "Pandas.json")
    
    if path.exists(new_filename):
        df = pd.read_json(new_filename)
    else:
        with open(orig_filename) as temp_json:
            temp_data = json.load(temp_json)
            temp_dict = create_song_dict()

            # Get all notes
            for note in temp_data["_notes"]:
                add_note_to_dict(temp_dict, note["_time"] * 60 / row["bpm"], note["_lineIndex"], note["_lineLayer"], 
                                 note["_type"], note["_cutDirection"])

            for event in temp_data["_events"]:
                add_event_to_dict(temp_dict, event["_time"] * 60 / row["bpm"], event["_type"], event["_value"])

            for obstacle in temp_data["_obstacles"]:
                add_obstacle_to_dict(temp_dict, obstacle["_time"] * 60 / row["bpm"], obstacle["_lineIndex"], obstacle["_type"],
                                    obstacle["_duration"], obstacle["_width"])

        # OUT OF SCOPE FOR THIS PROJECT FOR NOW
        #for bpm_change in temp_data["_BPMChanges"]
        df = pd.DataFrame(temp_dict)
        df["object_type"] = df["object_type"].astype('category')
        df.to_json(new_filename)
    return df

In [4]:
def download_song_with_key(key):
    try:
        new_song_df = get_single_song_metadata_df(key) # info from beatsaver
        download_song(key) # actual files from beatsaver
        new_song_df = get_song_info_from_local_files(new_song_df) # metadata that can only be found from local files
        new_song_df.apply(get_pandas_song_map, axis=1)
        
        return new_song_df
    except Exception as e:
        logger.warning("Couldn't download song for key: " + key + ", error is: " + str(e))
        empty_dict = {}
        empty_dict["key"] = [key]
        return pd.DataFrame(empty_dict)
    

def get_song_info_from_local_files(song_df):
    # Open the info.dat file which contains local metadata
    with open('./Data/{}/info.dat'.format(song_df.iloc[0]["key"]), errors="ignore", encoding="utf-8") as json_file:
        # Convert to JSON and grab the filename of the song
        data = json.load(json_file)
        song_df["filename"] = data["_songFilename"]
        
        # Get song length via librosa
        song_df["song_length"] = librosa.get_duration(filename="./Data/{}/{}".format(song_df.iloc[0]["key"], 
                                                                                         song_df.iloc[0]["filename"]))
        first_difficulty = True
        copy_df = song_df.copy()
        
        # Iterate through all types of maps
        for i in range(len(data["_difficultyBeatmapSets"])):
            beat_map_set = data["_difficultyBeatmapSets"][i]
            
            # We can only use standard, so make sure we have the standard map
            if beat_map_set["_beatmapCharacteristicName"] == "Standard":
                # Iterate through all different difficulties listed and grab their associated filename
                for j in range(len(beat_map_set["_difficultyBeatmaps"])):
                    dif_beat_map = beat_map_set["_difficultyBeatmaps"][j]
                    
                    if first_difficulty == True:
                        first_difficulty = False
                        song_df["difficulty"] = dif_beat_map["_difficulty"]
                        song_df["map_filename"] = dif_beat_map["_beatmapFilename"]
                    else:
                        copy_df["difficulty"] = dif_beat_map["_difficulty"]
                        copy_df["map_filename"] = dif_beat_map["_beatmapFilename"]
                        song_df = pd.concat([song_df, copy_df])
        
    return song_df
        
                                    
def get_single_song_metadata_df(data_key):
    """Retrieves the metadata info for a single song"""
    ratings_data = {}
    # Basic counts for upvotes, downvotes, and downloads and the download URL
    data = requests.get("https://beatsaver.com/api/maps/detail/{}".format(data_key)).json()
    
    logging.debug(data)
    
    ratings_data["key"] = [data_key]
    ratings_data["level_author_name"] = [data["metadata"]["levelAuthorName"]]
    ratings_data["song_name"] = [data["metadata"]["songName"]]
    ratings_data["song_sub_name"] = [data["metadata"]["songSubName"]]
    ratings_data["upvotes"] = [data["stats"]["upVotes"]]
    ratings_data["downvotes"] = [data["stats"]["downVotes"]]
    ratings_data["plays"] = [data["stats"]["plays"]]
    ratings_data["upload_date"] = [data["uploaded"]]
    ratings_data["downloads"] = [data["stats"]["downloads"]]
    ratings_data["bpm"] = [data["metadata"]["bpm"]]
    
    logging.info(ratings_data)
    song_df = pd.DataFrame.from_dict(ratings_data)
    song_df["net_votes"] = song_df["upvotes"] - song_df["downvotes"]
    song_df["percent_upvotes"] = 0
    
    if song_df.iloc[0]["upvotes"] != 0:
        song_df["percent_upvotes"] = song_df["upvotes"] / (song_df["upvotes"] + song_df["downvotes"])
    
    return song_df
    

def create_dir(key):
    """Creates a folder for the passed in key if one does not already exist. Returns the directory name"""
    directory = "./Data/{}".format(key)
    if not os.path.exists(directory):
        logging.debug("Path for " + key + " didn't already exist. Creating")
        os.makedirs(directory)
    return directory


def download_song(key):
    """Perform the actual song download"""
    directory = create_dir(key)

    if already_downloaded(key):
        logging.info("Songs already downloaded for key " + key)
    else:
        # Get the metadata and the song download link from the metadata
        song_zip = requests.get("https://beatsaver.com/api/download/key/{}".format(key), stream=True)
        os.chdir(directory)

        if song_zip.ok:
            # Extract to a zip file
            z = zipfile.ZipFile(BytesIO(song_zip.content))
            z.extractall()

            file_list = os.listdir(".")
            for file in file_list:
                # Clean up non-data files like cover, and random lightmap.exe
                if not file.endswith(".dat") and not file.endswith(".egg") and not file.endswith(".ogg"):
                    # There are also occasionally folders that are created. We don't need those either
                    if os.path.isfile(file):
                        os.remove(file)
                    else:
                        shutil.rmtree(file)
        else:
            logging.warning("Could not get files for key {}. Response code {}".format(key, song_zip.status_code))
            # Create a file in the folder showing that this key is invalid
            open("invalid.txt", "a").close()

        os.chdir("../..")


def already_downloaded(key):
    """Check if the song file has already been downloaded. Return true if it has"""
    lookup1 = "./Data/{}/*.egg".format(key)
    lookup2 = "./Data/{}/*.ogg".format(key)
    lookup3 = "./Data/{}/invalid.txt".format(key)
    return len(glob.glob(lookup1)) > 0 or len(glob.glob(lookup2)) > 0 or len(glob.glob(lookup3)) > 0

In [5]:
#%%time
# COMMENTED OUT SINCE THE FOLLOWING TOOK 4h 9min 51s to run for 25657 keys (with a lot of songs already downloaded)
# IF YOU RUN THE FOLLOWING IT WILL DOWNLOAD *ALL* BEATSABER SONGS ON THE COMMUNITY SITE
#logger = logging.getLogger()
#logger.setLevel(logging.CRITICAL)
#song_df = pd.DataFrame()

#max_key = int(requests.get("https://beatsaver.com/api/maps/latest/0").json()["docs"][0]["key"], 16)

# Get all hex keys
#for i in range(max_key):
#    new_song_df = download_song_with_key(hex(i)[2:])
#    song_df = pd.concat([song_df, new_song_df], sort=False)
    
#    if i % 100 == 0:
#        print("Downloaded {} Keys".format(i))

#song_df.to_csv("data/MetadataNew.csv")

Now that we have as many songs as we want, it is time to decide on a subset of songs to use for this

In [6]:
song_df = pd.read_csv("./Data/MetadataNew.csv")
song_df = song_df.sort_values("net_votes", ascending=False)
song_df.head(20)

,Unnamed: 0,key,level_author_name,song_name,song_sub_name,upvotes,downvotes,plays,upload_date,downloads,bpm,net_votes,percent_upvotes,filename,song_length,difficulty,map_filename
1996,0,570,greatyazer,Mr. Blue Sky,Electric Light Orchestra,6092.0,120.0,39426.0,2018-06-16T16:53:34.000Z,342795.0,174.0,5972.0,0.980683,song.egg,222.197551,Expert,Expert.dat
1994,0,570,greatyazer,Mr. Blue Sky,Electric Light Orchestra,6092.0,120.0,39426.0,2018-06-16T16:53:34.000Z,342795.0,174.0,5972.0,0.980683,song.egg,222.197551,Normal,Normal.dat
1995,0,570,greatyazer,Mr. Blue Sky,Electric Light Orchestra,6092.0,120.0,39426.0,2018-06-16T16:53:34.000Z,342795.0,174.0,5972.0,0.980683,song.egg,222.197551,Hard,Hard.dat
417,0,141,greatyazer,Gangnam Style,PSY,6164.0,225.0,82700.0,2018-05-20T09:59:02.000Z,520342.0,132.0,5939.0,0.964783,song.egg,218.824187,Normal,Normal.dat
418,0,141,greatyazer,Gangnam Style,PSY,6164.0,225.0,82700.0,2018-05-20T09:59:02.000Z,520342.0,132.0,5939.0,0.964783,song.egg,218.824187,Hard,Hard.dat
419,0,141,greatyazer,Gangnam Style,PSY,6164.0,225.0,82700.0,2018-05-20T09:59:02.000Z,520342.0,132.0,5939.0,0.964783,song.egg,218.824187,Expert,Expert.dat
377,0,124,jobas,Rasputin (Funk Overload),NaN,6253.0,318.0,29624.0,2018-05-19T16:36:34.000Z,406245.0,149.0,5935.0,0.951606,song.egg,238.524082,Hard,Hard.dat
596,0,1bf,calijor,Lone Digger,NaN,5569.0,287.0,57999.0,2018-05-23T00:15:19.000Z,356627.0,124.0,5282.0,0.950990,Caravan Palace - Lone Digger.egg,169.926531,Expert,Expert.dat
595,0,1bf,calijor,Lone Digger,NaN,5569.0,287.0,57999.0,2018-05-23T00:15:19.000Z,356627.0,124.0,5282.0,0.950990,Caravan Palace - Lone Digger.egg,169.926531,Hard,Hard.dat
594,0,1bf,calijor,Lone Digger,NaN,5569.0,287.0,57999.0,2018-05-23T00:15:19.000Z,356627.0,124.0,5282.0,0.950990,Caravan Palace - Lone Digger.egg,169.926531,Normal,Normal.dat


In [7]:
song_df["Key-Dif"] = song_df["key"] + "-" + song_df["difficulty"]
song_df.head(20)["Key-Dif"]

1996    570-Expert
1994    570-Normal
1995      570-Hard
417     141-Normal
418       141-Hard
419     141-Expert
377       124-Hard
596     1bf-Expert
595       1bf-Hard
594     1bf-Normal
593       1bf-Easy
1148    32e-Expert
1147      32e-Hard
736       217-Hard
735     217-Normal
734       217-Easy
737     217-Expert
294      e4-Expert
1423      3fc-Hard
1422    3fc-Normal
Name: Key-Dif, dtype: object

So we now have the keys that we want to use, but we need some actual data to train on. 

For training we are going to have 2 steps. The first step (which is started below) is to train a neural network on how to determine where to place notes, bombs, obstacles, and events in the song (without classifying the position or any classification problems about the objects themselves).

To do that the approach is as follows:
1. Create spectrograms of the song in a set size of chunks (e.g. 1 second chunks) so that we can have a constant input length (with the end of the song padded with 0s)
2. Get the count of each object type at each timestep (e.g. count how many notes occur at each 10 milisecond spot)
3. Use the spectrograms as the training data and the object counts as the "labels" for the neural net

In [8]:
def get_spectrogram_from_file(filename, num_mels, win_len, pitch_shift=0):
    y, sr = librosa.load(filename)
    
    if pitch_shift != 0:
        y = librosa.effects.pitch_shift(y, sr, pitch_shift)
        
    hop_length = 128
    
    if win_len < 10:
        hop_length = 64
        
    # Create a spectrogram with the window lengths that we want
    spec = librosa.feature.melspectrogram(y=y, sr=sr, 
                                          hop_length=hop_length, 
                                          n_mels=num_mels, fmin=20, fmax=8000)
    # Convert the power of the spectrogram into decibels instead and then normalize it around 1
    spec = librosa.power_to_db(spec, np.max, top_db=100) / 100 + 1
    
    file_len = librosa.core.get_duration(y, sr)
    
    time_indices = librosa.time_to_frames(np.arange(0, file_len, 0.001 * win_len), sr=sr, hop_length=hop_length)
    
    return spec, time_indices

def get_spectrogram(song_df, key, num_mels, win_len, pitch_shift=0):
    """Get the spectrograms of the song with the specified window length and number of mels"""
    
    # See if we already have the melspectrogram created since that can take a bit
    if pitch_shift == 0:
        spec_filename = "data/{}/{}-{}melspec.npy".format(key, num_mels, win_len)
    else:
        spec_filename = "data/{}/{}-{}-{}melspec.npy".format(key, num_mels, win_len, pitch_shift)
    
    index_filename = "data/{}/time_indices.npy".format(key)
    
    if path.exists(spec_filename):
        spec = np.load(spec_filename)
        time_indices = np.load(index_filename)
    else:
        # Get the name of file that will have the actual song data (e.g. song.ogg)
        filename = "data/{}/{}".format(key, song_df[song_df.key == key].iloc[0]["filename"])

        # Create the spectrogram
        spec, time_indices = get_spectrogram_from_file(filename, num_mels, win_len, pitch_shift)
        
        # Save the newly created file
        np.save(spec_filename, spec)
        
        if not path.exists(index_filename):
            np.save(index_filename, time_indices)
        
    return spec, time_indices

In [9]:
def set_onset_count_arr_from_df(df, object_type, arr, array_len):
    """Reads the data from the dataframe and adds it to the passed in numpy arr"""
    
    # Gets the current window that we are curious about
    temp_df = df[(df.object_type == object_type)]
    
    # Iterates
    for index, row in temp_df.iterrows():
        cur_index = int(row["input_num"])
        
        if cur_index < array_len:
            arr[cur_index] = row["time"]
        

def get_object_counts(song_df, key, difficulty, win_len, array_len):
    #import pdb; pdb.set_trace()
    
    object_filename = "data/{}/{}-{}objects.npy".format(key, difficulty, win_len)
    
    if path.exists(object_filename):
        object_arr = np.load(object_filename)
    else:
    
        # Get the filename and read it into a pandas dataframe
        obj_df = get_pandas_song_map(song_df[(song_df.key == key) & (song_df.difficulty == difficulty)].iloc[0])
        logging.debug("obj_df before: {}".format(obj_df.sort_values("time")))

        # Get the number of objects that occur at each timeframe
        obj_df["input_num"] = round(obj_df["time"] * 1000 / win_len)
        obj_df = obj_df.sort_values("input_num").groupby(["input_num", "object_type"])["time"].count().to_frame().reset_index()
        logging.debug("obj_df after: {}".format(obj_df))

        notes_arr = np.zeros(array_len)
        bombs_arr = np.zeros(array_len)
        obstacles_arr = np.zeros(array_len)
        events_arr = np.zeros(array_len)

        set_onset_count_arr_from_df(obj_df, "Note", notes_arr, array_len)
        set_onset_count_arr_from_df(obj_df, "Bomb", bombs_arr, array_len)
        set_onset_count_arr_from_df(obj_df, "Obstacle", obstacles_arr, array_len)
        set_onset_count_arr_from_df(obj_df, "Event", events_arr, array_len)
        
        object_arr = np.vstack([notes_arr, bombs_arr, obstacles_arr, events_arr])
        np.save(object_filename, object_arr)
    
    return object_arr

In [10]:
def dif_val_from_string(dif):
    # Get the difficulty as an integer. This is a category but we are NOT using one-hot encoding as
    # "Expert" is more like "ExpertPlus" than "Easy" so a numerical category is fine here
    # Normalized on 1 to match other neural net inputs
    if dif == "Easy":
        return .2
    elif dif == "Normal":
        return .4
    elif dif == "Hard":
        return .6
    elif dif == "Expert":
        return .8
    elif dif == "ExpertPlus":
        return 1
    else:
        raise Exception("Invalid difficulty passed to get_song_count_train_data")

def get_meta_arr(input_len, dif_val, win_len):
    dif_ray = np.ones((input_len)) * dif_val
        
    # Gets the end index of the first 10 seconds
    # End result should be an array that counts up to 1 over num_mil_start miliseconds
    num_mil_start = 10000
    start_indices = np.arange(0, input_len) * win_len / num_mil_start
    end_of_start = int(num_mil_start / win_len) 
    start_indices[end_of_start:] = start_indices[end_of_start]

    # Repeat for end 10 seconds
    num_mil_end = 10000
    num_end_indices = int(num_mil_end / win_len)
    start_of_end = input_len - num_end_indices
    end_indices = np.zeros(input_len)

    for i in range(num_end_indices):
        end_indices[start_of_end + i] = i * win_len / num_mil_end

    # Stack all of the meta data type arrays
    meta_arr = np.hstack([dif_ray.reshape(input_len, 1), 
                          start_indices.reshape(input_len, 1), 
                          end_indices.reshape(input_len, 1)])

    return meta_arr
        
def get_song_count_train_data(song_df, key, num_mels, win_len, difficulty, pitch_shift=0, obj_type="Note"):
    """Get lists of numpy arrays for a song with the training data and the counts (aka labels)"""
    
    # Get all spectrogram values for the song
    spec, time_indices = get_spectrogram(song_df, key, num_mels, win_len, pitch_shift)
    input_len = len(time_indices)
    
    obj_list = []
    
    # Get the difficulty value
    dif_val = dif_val_from_string(difficulty)
    
    if dif_val != 0:
        #import pdb; pdb.set_trace()
        meta_arr = get_meta_arr(input_len, dif_val, win_len)

        # Get the counts at each point for all objects
        objs = get_object_counts(song_df, key, difficulty, win_len, input_len)

        if obj_type == "Note":
            obj_arr = objs[0]
        elif obj_type == "Bomb":
            obj_arr = objs[1]
        elif obj_type == "Obst":
            obj_arr = objs[2]
        elif obj_type == "Event":
            obj_arr = objs[3]
        else:
            raise Exception("Invalid obj_type passed to get_song_count_train_data: {}".format(obj_type))
    
    
    return meta_arr, spec, time_indices, obj_arr

In [11]:
def chunk_spectrogram(arr, time_indices, start_index, num_samples, chunk_radius):
    """Converts a 2d array into a 3D array with different chunks. At a chunk radius of 5 each array index will include 5
    indices from both sides of every index of the array"""
    cur_index = int(start_index)
    
    #TODO - adjust to work with the new time index system
    
    chunk_arr_list = []
    
    while cur_index < min(len(time_indices), start_index + num_samples):
        if not isinstance(cur_index, int):
            logging.error("cur_index is not an integer. cur_index {}. Type {}".format(cur_index, type(cur_index)))
            
        true_index = time_indices[cur_index]
        
        min_index = int(max(0, true_index - chunk_radius))
        max_index = int(min(arr.shape[1], true_index + chunk_radius + 1))
        logging.debug("min_index is {} and max_index is {}".format(min_index, max_index))
        
        if min_index > max_index:
            logging.error("min_index is {} and max_index is {}".format(min_index, max_index))
            logging.error("true_index is {} and chunk_radius is {}, vs arr.shape[1] at {}".format(true_index, chunk_radius, arr.shape[1]))
        
        pad_left = int(abs(max(-(true_index - chunk_radius), 0)))
        pad_right = int(abs(min(0, arr.shape[1] - (true_index + chunk_radius + 1))))
        logging.debug("Pad_left is {} and pad_right is {}".format(pad_left, pad_right))
        #import pdb; pdb.set_trace()
        
        if pad_left > 0 or pad_right > 0:
            logging.debug("Padding from min index {} to max index {} with pad left {} and pad right {}".format(min_index, max_index, pad_left, pad_right))
            new_ray = np.pad(arr[:, min_index:max_index], [(0, 0), (pad_left, pad_right)], 'constant', constant_values=0)
        else:
            new_ray = arr[:, min_index:max_index]
            
        chunk_arr_list.append(new_ray)
        
        cur_index += 1
    
    if len(chunk_arr_list) == 0:
        logging.error("For array of shape {} at start index {} for num_samples {} got no values".format(arr.shape, start_index, num_samples))
    
    stack_arr = np.stack(chunk_arr_list)
    logging.debug("stack_arr shape: {}".format(stack_arr.shape))
    return stack_arr

In [12]:
def reshape_spec_arr(spec):
    # Reshapes an array of spectrogram
    N, ydim, xdim = spec.shape
    return spec.reshape(N, ydim, xdim, 1)

In [13]:
def set_if_in_range(arr, index, val):
    if index >= 0 and index < len(arr) and arr[index] < val:
        arr[index] = val

# Class for generating data to feed for note classification
class PlacementDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, key_difs, song_df, num_mels, win_len, chunk_radius, object_type="Note", batch_size=200):
        'Initialization'
        self.key_difs = key_difs
        self.song_df = song_df
        self.num_mels = num_mels
        self.win_len = win_len
        self.chunk_radius = chunk_radius
        self.object_type = object_type
        self.batch_size = batch_size
        self.num_samples = 0
        
        self.pitch_shift = 0
        self.pitch_index = 0
        self.pitch_shifts = [0, 4, -4, 8, -8, 12, -12, 16, -16]
        
        self.indices = []
        for key_dif in self.key_difs:
            key, dif = key_dif.split("-")
            spec, time_indices = get_spectrogram(song_df, key, num_mels, win_len)
            input_len = len(time_indices)
            
            self.num_samples += self.batch_size * math.ceil(input_len / self.batch_size)
            self.indices.append(self.num_samples)
        
        logging.info("Indices are: {}".format(self.indices))
        # self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.ceil(self.num_samples / self.batch_size)
    
    def on_epoch_end(self):
        'Update pitches after each epoc'
        
        self.pitch_index += 1
        
        if self.pitch_index >= len(self.pitch_shifts):
            self.pitch_index = 0
        
        self.pitch_shift = self.pitch_shifts[self.pitch_index]
        
        for key_dif in self.key_difs:
            key, dif = key_dif.split("-")
            spec, time_indices = get_spectrogram(song_df, key, num_mels, win_len, self.pitch_shift)
            
    
    def __getitem__(self, index):
        'Generate one batch of data'
        #import pdb; pdb.set_trace()
        # Get key and difficulty for current       
        key_index = 0
        
        logging.debug("Getting batch: {}".format(index))
        
        while index * self.batch_size >= self.indices[key_index]:
            key_index += 1
        
        key, dif = self.key_difs[key_index].split("-")
        
        if key_index > 0:
            index -= int(self.indices[key_index - 1] / self.batch_size)
        
        # Get data for the current key and difficulty
        meta_arr, spec_arr, time_indices, obj_arr = get_song_count_train_data(self.song_df, key, self.num_mels, self.win_len, dif, 
                                                                self.pitch_shift, self.object_type)
        logging.debug("Index is: {}".format(index))
        spec_arr_new = chunk_spectrogram(spec_arr, time_indices, index * self.batch_size, self.batch_size, self.chunk_radius)
        
        # We are only looking at whether a note is at a timestep or not, so reduce to either 1 or 0
        obj_arr = np.ceil(obj_arr / 100)
        
        logging.debug("{} array {}".format(self.object_type, obj_arr))
        end_index = min(meta_arr.shape[0], index * self.batch_size + self.batch_size)
        
        # We care about song index because it has to learn to not play notes at the very beginning and end of the song
        meta_arr = meta_arr[index * self.batch_size:end_index]
        
        obj_arr = obj_arr[index * self.batch_size:end_index]
        
        set_indices = []
        
        for i in range(0, len(obj_arr)):
            if obj_arr[i] == 1:
                set_indices.append(i - 1)
                set_indices.append(i + 1)
        
        for index in set_indices:
            set_if_in_range(obj_arr, index, 1)
        
        logging.debug("{} array after updates: {}".format(self.object_type, obj_arr))
        
        # Reshape the spectrogram to fit the expected 4 dimensions
        spec_arr_new = reshape_spec_arr(spec_arr_new)
        
        logging.debug("Shapes. Spec: {}, Dif: {}, Note: {}".format(spec_arr_new.shape, meta_arr.shape, obj_arr.shape))
        
        return [spec_arr_new, meta_arr], obj_arr

Keras model

In [14]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [15]:
# For cleaning up misc numpy files if needed
#for filename in glob.glob("Data/*/*.npy"):
#    os.remove(filename)

In [16]:
def get_experiment_number(model_folder):
    if not path.exists("Experiments/{}/experiment_count.txt".format(model_folder)):
        return 1
    else:
        with open("Experiments/{}/experiment_count.txt".format(model_folder), "r") as f:
            num = f.read()
        return int(num)

In [17]:
def write_experiment_number(model_folder, num):
    with open("Experiments/{}/experiment_count.txt".format(model_folder), "w") as f:
        f.write(str(num))

In [18]:
def increment_experiment_number(model_folder):
    num = get_experiment_number(model_folder)
    write_experiment_number(model_folder, num + 1)

In [19]:
def get_experiment_folder(model_folder):
    exp_num = get_experiment_number(model_folder)
    
    file_path = "Experiments/{}/Experiment{}".format(model_folder, exp_num)
    
    if not path.exists(file_path):
        os.mkdir(file_path)
    
    return file_path

In [20]:
def get_experiment_dfs(model_folder):
    summary_filename = "Experiments/{}/model_summaries.pd".format(model_folder)
    if not path.exists(summary_filename):
        
        model_dict = {"experiment_num": [], "valid": [], "num_mels": [], "win_len": [], "chunk_radius": [], 
            "batch_size": [], "learning_rate": [], "note_class_weight": [],
            "model_version": [], "spec_conv_layers": [], "spec_dense_layers": [], "combined_layers": []}
        
        model_df = pd.DataFrame(model_dict)
        model_df.to_csv(summary_filename, index=False)
    else:
        model_df = pd.read_csv(summary_filename)
        
    metric_filename = "Experiments/{}/training_metrics.pd".format(model_folder)
    if not path.exists(metric_filename):
        
        metric_dict = {"val_loss": [], "val_acc":[], "val_precision":[], "val_recall": [],
            "loss": [], "acc":[], "precision":[], "recall": [],
            "epoch_num": [], "experiment_num": []}
        
        metric_df = pd.DataFrame(metric_dict)
        metric_df.to_csv(metric_filename, index=False)
    else:
        metric_df = pd.read_csv(metric_filename)
    
    return model_df, metric_df

In [21]:
def add_dense_layers(input_layer, units, activation="relu", num_layers=1):
    dense = Dense(units, activation ='relu')(input_layer)
    
    for i in range(num_layers - 1):
        dense = Dense(units, activation ='relu')(dense)
        
    return dense

In [22]:
def add_conv2D_layers(input_layer, filters, kernel_size, num_1x1_layers = 0, max_pool=False, activation="relu"):
    conv = Convolution2D(filters, kernel_size, activation=activation)(input_layer)
    
    for i in range(num_1x1_layers):
        conv = Convolution2D(filters, (1,1), activation=activation)(conv)
    
    if max_pool == True:
        conv = MaxPooling2D(pool_size=(2,2))(conv)
        
    return conv

In [23]:
def create_conv2D_obj(filters, kernel_size, num_1x1_layers = 0, max_pool=False, activation="relu"):
    return {"filters":filters, "kernel_size": kernel_size, 
            "layers": num_1x1_layers, "max_pool": max_pool, "activation": activation}

In [24]:
def create_dense_obj(units, activation="relu", num_layers=1):
    return {"units":units, "activation":activation, "num_layers":num_layers}

In [25]:
def run_note_place_experiment(song_df, num_mels, win_len, chunk_radius, 
                              batch_size, num_epochs, learning_rate, 
                              note_class_weight, train_keys, validation_keys,
                              model_folder, model_version, 
                              spec_conv_layers, spec_dense_layers, combined_layers):
    
    exp_num = get_experiment_number(model_folder)
    summary_df, metric_df = get_experiment_dfs(model_folder)
    
    model_dict = {"experiment_num": [exp_num], "valid": [True],  "num_mels": [num_mels], "win_len": [win_len], "chunk_radius": [chunk_radius], 
            "batch_size": [batch_size], "learning_rate": [learning_rate], "note_class_weight": [note_class_weight],
            "model_version": [model_version], "spec_conv_layers": [spec_conv_layers],
            "spec_dense_layers": [spec_dense_layers], "combined_layers": [combined_layers]}
    
    model_df = pd.DataFrame(model_dict)
    
    model_dup_df = model_df.drop("experiment_num", axis=1)
    summary_dup_df = summary_df.drop("experiment_num", axis=1)
    
    count_row = summary_df.shape[0]
    model_string = model_dup_df.iloc[0].to_string()
    
    is_duplicate = False

    for i in range(count_row):
        if summary_dup_df.iloc[i].to_string() == model_string:
            is_duplicate = True
            break
    
    
    # Check for duplicates
    if is_duplicate == False:
        summary_df = pd.concat([summary_df, model_df])
        
        valid_model = True
        try:
            image_input = Input(shape=(num_mels, chunk_radius * 2 + 1, 1))

            first = True

            for conv_layer in spec_conv_layers:
                if first:
                    conv = add_conv2D_layers(image_input, conv_layer["filters"], conv_layer["kernel_size"],
                                               conv_layer["layers"], conv_layer["max_pool"], conv_layer["activation"])
                    first = False
                else:
                    conv = add_conv2D_layers(conv, conv_layer["filters"], conv_layer["kernel_size"],
                                               conv_layer["layers"], conv_layer["max_pool"], conv_layer["activation"])

            # Flatten for output to a dense layer
            if first:
                first_dense = Flatten()(image_input)
            else:
                first_dense = Flatten()(conv)


            # Add any dense layers that are defined
            for dense_layer in spec_dense_layers:
                first_dense = add_dense_layers(first_dense, dense_layer["units"], dense_layer["activation"], dense_layer["num_layers"])


            # Timing and difficulty information
            other_data_input = Input(shape=(3, ))

            merged_model = keras.layers.concatenate([first_dense, other_data_input])

            for dense_layer in combined_layers:
                merged_model = add_dense_layers(merged_model, dense_layer["units"], dense_layer["activation"], dense_layer["num_layers"])

            predictions = Dense(1, activation ='sigmoid')(merged_model)

            # Create the model
            note_place_class_model = Model(inputs=[image_input, other_data_input], outputs=predictions)

            # Set optimizer and compile
            optimizer = keras.optimizers.Adam(lr=learning_rate)
            note_place_class_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy", keras_metrics.precision(), keras_metrics.recall()])

            # Create generators
            train_generator = PlacementDataGenerator(train_keys, song_df, num_mels, win_len, chunk_radius, "Note", batch_size)
            validation_generator = PlacementDataGenerator(validation_keys, song_df, num_mels, win_len, chunk_radius, "Note", batch_size)


            history = keras.callbacks.History()
            note_place_class_model.fit_generator(generator=train_generator, 
                                                 epochs=num_epochs, 
                                                 validation_data=validation_generator, 
                                                 class_weight=note_class_weight,
                                                 callbacks=[history])
        except ValueError as e:
            summary_df.loc[summary_df.experiment_num == exp_num, "valid"] = False
            valid_model = False

            logging.error("Experiment skipped due to error. Model: {}\nError: {}".format(model_dict, e))


        exp_folder = get_experiment_folder(model_folder)

        if valid_model:
            hist_df = pd.DataFrame(history.history)
            hist_df['epoch_num'] = hist_df.index + 1
            hist_df['experiment_num'] = exp_num

            hist_df.to_csv("{}/hist.pd".format(exp_folder), index=False)

            metric_df = pd.concat([metric_df, hist_df])

            metric_filename = "Experiments/{}/training_metrics.pd".format(model_folder)
            metric_df.to_csv(metric_filename, index=False)

        summary_filename = "Experiments/{}/model_summaries.pd".format(model_folder)
        summary_df.to_csv(summary_filename, index=False)

        increment_experiment_number(model_folder)
    else:
        logging.info("Experiment skipped due to duplicates. Model: {}".format(model_dict))

In [26]:
train_keys_xxl = song_df.head(100)["Key-Dif"].to_numpy()[0:90]
validation_keys_xxl = song_df.head(100)["Key-Dif"].to_numpy()[90:100]
train_keys_xl = song_df.head(50)["Key-Dif"].to_numpy()[0:41]
validation_keys_xl = song_df.head(50)["Key-Dif"].to_numpy()[41:50]
train_keys_l = song_df.head(20)["Key-Dif"].to_numpy()[0:13]
validation_keys_l = song_df.head(20)["Key-Dif"].to_numpy()[13:20]
train_keys_m = song_df.head(20)["Key-Dif"].to_numpy()[0:3]
validation_keys_m = song_df.head(20)["Key-Dif"].to_numpy()[4:6]
train_keys_s = [song_df.head(20)["Key-Dif"].to_numpy()[0]]
validation_keys_s = [song_df.head(20)["Key-Dif"].to_numpy()[5]]

In [27]:
num_mels = 80
win_len = 10
batch_size = 400
num_epochs = 50

#chunk_radius = 5
#learning_rate = 0.0001

#class_weight = {0: 1, .25: 5, .5: 5, .75: 5, .9: 5, 1: 10} # Weigh onsets more heavily than non-onsets due to the imbalance
#note_class_weight = {0: 1, 1: 15} # Weigh onsets more heavily than non-onsets due to the imbalance
#event_class_weight = {0: 1, 1: 15}

train_keys = train_keys_l
validation_keys = validation_keys_l
model_folder = "NotePlacement"
model_version = 1 # Since this is the first version since we have started logging

In [28]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

for chunk_radius in [3, 5, 7, 9, 11]:
#for chunk_radius in [5]:
    for learning_rate in [0.00001, 0.0001, 0.001]:
    #for learning_rate in [0.001]:
        for note_weight in [10, 15, 20]:
        #for note_weight in [15]:
            note_class_weight = {0: 1, 1: note_weight}
            
            for i in [1, 2, 3]:
                spec_conv_layers = []
                spec_dense_layers = []
                combined_layers = []
                
                if i == 1:
                    spec_conv_layers.append(create_conv2D_obj(64, (1, chunk_radius * 2 + 1), 4, False, "relu"))
                    
                    spec_dense_layers.append(create_dense_obj(10 * chunk_radius * 2 + 1, "relu", 3))
                    
                    combined_layers.append(create_dense_obj(chunk_radius * 2 + 1, "relu", 3))
                elif i == 2:
                    spec_conv_layers.append(create_conv2D_obj(64, (3, 3), 2, True, "relu"))
                    spec_conv_layers.append(create_conv2D_obj(128, (3, 3), 2, True, "relu"))
                    
                    spec_dense_layers.append(create_dense_obj(10 * chunk_radius * 2 + 1, "relu", 2))
                    
                    combined_layers.append(create_dense_obj(chunk_radius * 2 + 1, "relu", 3))
                elif i == 3:
                    spec_conv_layers.append(create_conv2D_obj(64, (3, 3), 2, True, "relu"))
                    spec_conv_layers.append(create_conv2D_obj(128, (3, 3), 2, True, "relu"))
                    spec_conv_layers.append(create_conv2D_obj(256, (3, 3), 2, True, "relu"))
                    spec_conv_layers.append(create_conv2D_obj(512, (3, 3), 2, True, "relu"))
                    
                    combined_layers.append(create_dense_obj(chunk_radius * 2 + 1, "relu", 3))
                    
                run_note_place_experiment(song_df, num_mels, win_len, chunk_radius, 
                              batch_size, num_epochs, learning_rate, 
                              note_class_weight, train_keys, validation_keys,
                              model_folder, model_version, 
                              spec_conv_layers, spec_dense_layers, combined_layers)

INFO:root:Experiment skipped due to duplicates. Model: {'experiment_num': [240], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [3], 'batch_size': [400], 'learning_rate': [1e-05], 'note_class_weight': [{0: 1, 1: 10}], 'model_version': [1], 'spec_conv_layers': [[{'filters': 64, 'kernel_size': (1, 7), 'layers': 4, 'max_pool': False, 'activation': 'relu'}]], 'spec_dense_layers': [[{'units': 61, 'activation': 'relu', 'num_layers': 3}]], 'combined_layers': [[{'units': 7, 'activation': 'relu', 'num_layers': 3}]]}



ERROR:root:Experiment skipped due to error. Model: {'experiment_num': [240], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [3], 'batch_size': [400], 'learning_rate': [1e-05], 'note_class_weight': [{0: 1, 1: 10}], 'model_version': [1], 'spec_conv_layers': [[{'filters': 64, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}, {'filters': 128, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}]], 'spec_dense_layers': [[{'units': 61, 'activation': 'relu', 'num_layers': 2}]], 'combined_layers': [[{'units': 7, 'activation': 'relu', 'num_layers': 3}]]}
Error: Negative dimension size caused by subtracting 3 from 2 for 'conv2d_4/convolution' (op: 'Conv2D') with input shapes: [?,39,2,64], [3,3,64,128].
ERROR:root:Experiment skipped due to error. Model: {'experiment_num': [241], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [3], 'batch_size': [400], 'learning_rate': [1e-05], 'note_class_weight': [{0: 1, 

INFO:root:Experiment skipped due to duplicates. Model: {'experiment_num': [248], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [3], 'batch_size': [400], 'learning_rate': [0.0001], 'note_class_weight': [{0: 1, 1: 15}], 'model_version': [1], 'spec_conv_layers': [[{'filters': 64, 'kernel_size': (1, 7), 'layers': 4, 'max_pool': False, 'activation': 'relu'}]], 'spec_dense_layers': [[{'units': 61, 'activation': 'relu', 'num_layers': 3}]], 'combined_layers': [[{'units': 7, 'activation': 'relu', 'num_layers': 3}]]}
ERROR:root:Experiment skipped due to error. Model: {'experiment_num': [248], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [3], 'batch_size': [400], 'learning_rate': [0.0001], 'note_class_weight': [{0: 1, 1: 15}], 'model_version': [1], 'spec_conv_layers': [[{'filters': 64, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}, {'filters': 128, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}

INFO:root:Experiment skipped due to duplicates. Model: {'experiment_num': [256], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [3], 'batch_size': [400], 'learning_rate': [0.001], 'note_class_weight': [{0: 1, 1: 20}], 'model_version': [1], 'spec_conv_layers': [[{'filters': 64, 'kernel_size': (1, 7), 'layers': 4, 'max_pool': False, 'activation': 'relu'}]], 'spec_dense_layers': [[{'units': 61, 'activation': 'relu', 'num_layers': 3}]], 'combined_layers': [[{'units': 7, 'activation': 'relu', 'num_layers': 3}]]}
ERROR:root:Experiment skipped due to error. Model: {'experiment_num': [256], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [3], 'batch_size': [400], 'learning_rate': [0.001], 'note_class_weight': [{0: 1, 1: 20}], 'model_version': [1], 'spec_conv_layers': [[{'filters': 64, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}, {'filters': 128, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}]]

INFO:root:Experiment skipped due to duplicates. Model: {'experiment_num': [261], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [5], 'batch_size': [400], 'learning_rate': [0.0001], 'note_class_weight': [{0: 1, 1: 10}], 'model_version': [1], 'spec_conv_layers': [[{'filters': 64, 'kernel_size': (1, 11), 'layers': 4, 'max_pool': False, 'activation': 'relu'}]], 'spec_dense_layers': [[{'units': 101, 'activation': 'relu', 'num_layers': 3}]], 'combined_layers': [[{'units': 11, 'activation': 'relu', 'num_layers': 3}]]}
INFO:root:Experiment skipped due to duplicates. Model: {'experiment_num': [261], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [5], 'batch_size': [400], 'learning_rate': [0.0001], 'note_class_weight': [{0: 1, 1: 10}], 'model_version': [1], 'spec_conv_layers': [[{'filters': 64, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}, {'filters': 128, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 

...

Epoch 39/50
679/679 [==============================] - 20s 30ms/step - loss: 1.1806 - acc: 0.6297 - precision: 0.1315 - recall: 0.8206 - val_loss: 0.3933 - val_acc: 0.8345 - val_precision: 0.1508 - val_recall: 0.4304
Epoch 40/50
679/679 [==============================] - 20s 29ms/step - loss: 1.1337 - acc: 0.6561 - precision: 0.1402 - recall: 0.8180 - val_loss: 0.5005 - val_acc: 0.7610 - val_precision: 0.1336 - val_recall: 0.6062
Epoch 41/50
679/679 [==============================] - 20s 30ms/step - loss: 1.2190 - acc: 0.6191 - precision: 0.1270 - recall: 0.8109 - val_loss: 0.5637 - val_acc: 0.7227 - val_precision: 0.1158 - val_recall: 0.6052
Epoch 42/50
679/679 [==============================] - 20s 30ms/step - loss: 1.0683 - acc: 0.6750 - precision: 0.1489 - recall: 0.8300 - val_loss: 0.5846 - val_acc: 0.6688 - val_precision: 0.1058 - val_recall: 0.6702
Epoch 43/50
679/679 [==============================] - 20s 30ms/step - loss: 1.2602 - acc: 0.5718 - precision: 0.1149 - recall: 0.81

INFO:root:Indices are: [22400, 44800, 67200, 89200, 111200, 133200, 157200, 174400, 191600, 208800, 226000, 248800, 271600]
INFO:root:Indices are: [14400, 28800, 43200, 57600, 77600, 101200, 124800]


Epoch 1/50
679/679 [==============================] - 91s 134ms/step - loss: 1.4707 - acc: 0.2264 - precision: 0.0743 - recall: 0.9226 - val_loss: 0.7272 - val_acc: 0.6170 - val_precision: 0.0970 - val_recall: 0.7134
Epoch 2/50
679/679 [==============================] - 90s 132ms/step - loss: 1.2869 - acc: 0.6583 - precision: 0.1311 - recall: 0.7407 - val_loss: 0.8185 - val_acc: 0.5852 - val_precision: 0.0928 - val_recall: 0.7412
Epoch 3/50
679/679 [==============================] - 90s 132ms/step - loss: 1.2858 - acc: 0.6472 - precision: 0.1272 - recall: 0.7398 - val_loss: 0.6703 - val_acc: 0.8155 - val_precision: 0.1463 - val_recall: 0.4837
Epoch 4/50
679/679 [==============================] - 89s 131ms/step - loss: 1.2425 - acc: 0.7024 - precision: 0.1453 - recall: 0.7173 - val_loss: 0.6495 - val_acc: 0.7879 - val_precision: 0.1472 - val_recall: 0.5922
Epoch 5/50
679/679 [==============================] - 90s 132ms/step - loss: 1.3096 - acc: 0.6305 - precision: 0.1231 - recall: 0.74

Epoch 39/50
679/679 [==============================] - 90s 132ms/step - loss: 1.0945 - acc: 0.6549 - precision: 0.1414 - recall: 0.8310 - val_loss: 0.5115 - val_acc: 0.7528 - val_precision: 0.1203 - val_recall: 0.5503
Epoch 40/50
679/679 [==============================] - 90s 132ms/step - loss: 1.0319 - acc: 0.6978 - precision: 0.1590 - recall: 0.8327 - val_loss: 0.5047 - val_acc: 0.7652 - val_precision: 0.1285 - val_recall: 0.5616
Epoch 41/50
679/679 [==============================] - 90s 132ms/step - loss: 1.1475 - acc: 0.6462 - precision: 0.1361 - recall: 0.8139 - val_loss: 0.5311 - val_acc: 0.6961 - val_precision: 0.0992 - val_recall: 0.5569
Epoch 42/50
679/679 [==============================] - 89s 132ms/step - loss: 1.0176 - acc: 0.6917 - precision: 0.1580 - recall: 0.8454 - val_loss: 0.5806 - val_acc: 0.6950 - val_precision: 0.1038 - val_recall: 0.5919
Epoch 43/50
679/679 [==============================] - 90s 132ms/step - loss: 1.1914 - acc: 0.6150 - precision: 0.1268 - recall:

ERROR:root:Experiment skipped due to error. Model: {'experiment_num': [308], 'valid': [True], 'num_mels': [80], 'win_len': [10], 'chunk_radius': [11], 'batch_size': [400], 'learning_rate': [0.001], 'note_class_weight': [{0: 1, 1: 20}], 'model_version': [1], 'spec_conv_layers': [[{'filters': 64, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}, {'filters': 128, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}, {'filters': 256, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}, {'filters': 512, 'kernel_size': (3, 3), 'layers': 2, 'max_pool': True, 'activation': 'relu'}]], 'spec_dense_layers': [[]], 'combined_layers': [[{'units': 23, 'activation': 'relu', 'num_layers': 3}]]}
Error: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_452/convolution' (op: 'Conv2D') with input shapes: [?,8,1,256], [3,3,256,512].


In [29]:
summary_df, metric_df = get_experiment_dfs(model_folder)
summary_df

,experiment_num,valid,num_mels,win_len,chunk_radius,batch_size,learning_rate,note_class_weight,model_version,spec_conv_layers,spec_dense_layers,combined_layers
0,1,True,80,10,3,400,0.00001,"{0: 1, 1: 10}",1,"[{'filters': 64, 'kernel_size': (1, 7), 'layer...","[{'units': 61, 'activation': 'relu', 'num_laye...","[{'units': 7, 'activation': 'relu', 'num_layer..."
1,2,False,80,10,3,400,0.00001,"{0: 1, 1: 10}",1,"[{'filters': 64, 'kernel_size': (3, 3), 'layer...","[{'units': 61, 'activation': 'relu', 'num_laye...","[{'units': 7, 'activation': 'relu', 'num_layer..."
2,3,False,80,10,3,400,0.00001,"{0: 1, 1: 10}",1,"[{'filters': 64, 'kernel_size': (3, 3), 'layer...",[],"[{'units': 7, 'activation': 'relu', 'num_layer..."
3,4,True,80,10,3,400,0.00001,"{0: 1, 1: 15}",1,"[{'filters': 64, 'kernel_size': (1, 7), 'layer...","[{'units': 61, 'activation': 'relu', 'num_laye...","[{'units': 7, 'activation': 'relu', 'num_layer..."
4,5,False,80,10,3,400,0.00001,"{0: 1, 1: 15}",1,"[{'filters': 64, 'kernel_size': (3, 3), 'layer...","[{'units': 61, 'activation': 'relu', 'num_laye...","[{'units': 7, 'activation': 'relu', 'num_layer..."
5,6,False,80,10,3,400,0.00001,"{0: 1, 1: 15}",1,"[{'filters': 64, 'kernel_size': (3, 3), 'layer...",[],"[{'units': 7, 'activation': 'relu', 'num_layer..."
6,7,True,80,10,3,400,0.00001,"{0: 1, 1: 20}",1,"[{'filters': 64, 'kernel_size': (1, 7), 'layer...","[{'units': 61, 'activation': 'relu', 'num_laye...","[{'units': 7, 'activation': 'relu', 'num_layer..."
7,8,False,80,10,3,400,0.00001,"{0: 1, 1: 20}",1,"[{'filters': 64, 'kernel_size': (3, 3), 'layer...","[{'units': 61, 'activation': 'relu', 'num_laye...","[{'units': 7, 'activation': 'relu', 'num_layer..."
8,9,False,80,10,3,400,0.00001,"{0: 1, 1: 20}",1,"[{'filters': 64, 'kernel_size': (3, 3), 'layer...",[],"[{'units': 7, 'activation': 'relu', 'num_layer..."
9,10,True,80,10,3,400,0.00010,"{0: 1, 1: 10}",1,"[{'filters': 64, 'kernel_size': (1, 7), 'layer...","[{'units': 61, 'activation': 'relu', 'num_laye...","[{'units': 7, 'activation': 'relu', 'num_layer..."


In [47]:
metric_df["val_acc_val_prec_val_recall"] = metric_df["val_acc"] + metric_df["val_precision"] + metric_df["val_recall"]
metric_df.sort_values("val_loss", ascending=True)

,val_loss,val_acc,val_precision,val_recall,loss,acc,precision,recall,epoch_num,experiment_num,val_acc_val_prec,val_acc_val_prec_val_recall
153,0.274599,0.922604,0.160080,0.094276,0.906488,0.830758,0.213530,0.572336,4,10,1.082683,1.176960
1664,0.278654,0.912960,0.252427,0.293076,0.858906,0.844002,0.236435,0.599799,15,158,1.165387,1.458463
328,0.281642,0.917652,0.227487,0.204509,0.840107,0.840533,0.228663,0.585371,29,19,1.145140,1.349649
3510,0.287094,0.897248,0.207340,0.304348,0.827327,0.854692,0.252228,0.599131,11,292,1.104588,1.408936
3519,0.287145,0.902304,0.205658,0.268189,0.790889,0.864024,0.269589,0.606819,20,292,1.107962,1.376151
3470,0.288091,0.905274,0.168186,0.180940,0.846915,0.847672,0.236334,0.574676,21,291,1.073460,1.254400
3486,0.294346,0.890209,0.190446,0.303469,0.745943,0.859094,0.267247,0.637903,37,291,1.080654,1.384124
1141,0.306713,0.890516,0.160365,0.231591,0.728936,0.844221,0.249309,0.663306,42,87,1.050881,1.282472
3596,0.306795,0.882984,0.170352,0.288684,0.887864,0.762668,0.189690,0.781182,47,294,1.053336,1.342020
187,0.307306,0.893534,0.153678,0.205534,0.822088,0.852862,0.248007,0.592725,38,10,1.047212,1.252745


In [50]:
summary_df[summary_df.experiment_num == 293]

,experiment_num,valid,num_mels,win_len,chunk_radius,batch_size,learning_rate,note_class_weight,model_version,spec_conv_layers,spec_dense_layers,combined_layers
292,293,False,80,10,11,400,0.0001,"{0: 1, 1: 10}",1,"[{'filters': 64, 'kernel_size': (3, 3), 'layer...",[],"[{'units': 23, 'activation': 'relu', 'num_laye..."


In [54]:
summary_df[summary_df.experiment_num == 293]["spec_conv_layers"]

292    [{'filters': 64, 'kernel_size': (3, 3), 'layer...
Name: spec_conv_layers, dtype: object


In [48]:
metric_df[metric_df.experiment_num == 294].sort_values("epoch_num", ascending=False)

,val_loss,val_acc,val_precision,val_recall,loss,acc,precision,recall,epoch_num,experiment_num,val_acc_val_prec,val_acc_val_prec_val_recall
3599,0.591798,0.714890,0.097825,0.505782,0.968682,0.732792,0.168847,0.765696,50,294,0.812716,1.318498
3598,0.432596,0.807659,0.125144,0.413995,0.857624,0.771223,0.199077,0.803019,49,294,0.932803,1.346798
3597,0.447922,0.803516,0.111823,0.368028,0.919202,0.749067,0.180927,0.782408,48,294,0.915339,1.283367
3596,0.306795,0.882984,0.170352,0.288684,0.887864,0.762668,0.189690,0.781182,47,294,1.053336,1.342020
3595,0.309137,0.879286,0.178574,0.328942,0.878430,0.770494,0.195555,0.783188,46,294,1.057860,1.386802
3594,0.701081,0.577930,0.082572,0.656419,1.126397,0.645033,0.126066,0.727926,45,294,0.660502,1.316922
3593,0.457902,0.807869,0.132804,0.447811,0.872642,0.774216,0.198205,0.782463,44,294,0.940673,1.388484
3592,0.466899,0.799107,0.097644,0.319719,1.033254,0.701571,0.151631,0.755445,43,294,0.896751,1.216470
3591,0.507086,0.767108,0.125026,0.535793,0.877324,0.773655,0.197306,0.779344,42,294,0.892134,1.427926
3590,0.496189,0.765296,0.095587,0.383692,1.010813,0.727613,0.162904,0.744471,41,294,0.860883,1.244575


In [ ]:
class CountDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, key_difs, song_df, num_mels, win_len, chunk_radius, 
                 object_type="Note", max_placements=13, batch_size=200, 
                 count_weights={1: 1, 2: 1, 3: 100, 4: 50, 5: 500, 6: 400}):
        'Initialization'
        self.key_difs = key_difs
        self.song_df = song_df
        self.num_mels = num_mels
        self.win_len = win_len
        self.chunk_radius = chunk_radius
        self.object_type = object_type
        self.max_placements = max_placements
        self.batch_size = batch_size
        self.count_weights = count_weights
        
        self.pitch_shift = 0
        self.pitch_index = 0
        self.pitch_shifts = [0, 4, -4, 8, -8, 12, -12, 16, -16]

        self.val_counts = {}
        
        for i in range(self.max_placements):
            self.val_counts[i] = 0
        
        self.get_data_for_epoch()
        self.num_samples = self.specs.shape[0]
        logging.debug("Objs: {}".format(self.objs))
        # self.on_epoch_end()

    def get_data_for_epoch(self):
        spec_list = []
        meta_list = []
        obj_list = []
        
        for key_dif in self.key_difs:
            key, dif = key_dif.split("-")
            
            meta_arr, spec, time_indices, obj_arr = get_song_count_train_data(self.song_df, key, self.num_mels, self.win_len, dif, 
                                                                self.pitch_shift, self.object_type)
            
            
            for index in np.nonzero(obj_arr)[0]:
                spec_arr_new = chunk_spectrogram(spec, time_indices, index, 1, self.chunk_radius)
                spec_list.append(spec_arr_new)
                meta_list.append(meta_arr[index, 0]) # We only care about difficulty for this, so get only that column
                obj_list.append(obj_arr[index])
        
        
        self.specs = np.vstack(spec_list)
        self.specs = reshape_spec_arr(self.specs)
        self.meta = np.stack(meta_list)
        self.objs = keras.utils.to_categorical(np.stack(obj_list), self.max_placements)
        logging.debug("Shapes for epoch. Specs: {}, meta: {}, objs: {}".format(self.specs.shape, 
                                                                              self.meta.shape, self.objs.shape))
    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.ceil(self.num_samples / self.batch_size)
    
    def on_epoch_end(self):
        'Update pitches after each epoc'
        
        self.pitch_index += 1
        
        if self.pitch_index >= len(self.pitch_shifts):
            self.pitch_index = 0
        
        self.pitch_shift = self.pitch_shifts[self.pitch_index]
        self.get_data_for_epoch()
        
        logging.debug("Value counts: {}".format(self.val_counts))
        self.val_counts = {}
        
        for i in range(self.max_placements):
            self.val_counts[i] = 0
    
    def __getitem__(self, index):
        'Generate one batch of data'
        #import pdb; pdb.set_trace()
        # Get key and difficulty for current         
        x = [self.specs[index:index+batch_size], self.meta[index:index+batch_size]]
        y = self.objs[index:index+batch_size]
        
        sample_weights = np.ones(y.shape[0])
        logging.debug("Y values?: {}".format(y))
        
        for i in range(2, max_placements):
            
            non_zero = np.nonzero(y[:, i])[0]
            
            for sub_index in non_zero:
                if i in self.count_weights:
                    sample_weights[sub_index] = self.count_weights[i]
                else:
                    sample_weights[sub_index] = i * 10
                
                self.val_counts[i] += 1
            
        logging.debug("Sample weights: {}".format(sample_weights))
        return x, y, sample_weights

In [ ]:
train_keys = train_keys_s
validation_keys = validation_keys_s
max_placements = 13

In [ ]:
# Spectrogram input
# PREDICTING NOTES COUNTS
image_input = Input(shape=(num_mels, chunk_radius * 2 + 1, 1))

# Timing and difficulty information
other_data_input = Input(shape=(1, ))   

# First convolution
conv = Convolution2D(64, (3,3), activation="relu")(image_input)
conv = Convolution2D(64, (1,1), activation="relu")(conv)
conv = Convolution2D(64, (1,1), activation="relu")(conv)
conv = MaxPooling2D(pool_size=(2,2))(conv)

# Second Convolution
conv = Convolution2D(128, (3,3), activation="relu")(image_input)
conv = Convolution2D(128, (1,1), activation="relu")(conv)
conv = Convolution2D(128, (1,1), activation="relu")(conv)
conv = MaxPooling2D(pool_size=(2,2))(conv)

# Flatten the output to enable the merge to happen with the other input
first_part_output = Flatten()(conv)

# Merge the output of the convNet with the added features by concatenation
merged_model = keras.layers.concatenate([first_part_output, other_data_input])

# Add an extra dense layer between the data and the output
dense = Dense(chunk_radius * 2 + 1, activation ='relu')(merged_model)
#dense = Dropout(.2)(dense)
dense = Dense(chunk_radius * 2 + 1, activation ='relu')(dense)
#dense = Dropout(.2)(dense)
dense = Dense(chunk_radius * 2 + 1, activation ='relu')(dense)
#dense = Dropout(.2)(dense)

# Predict on the output 
predictions = Dense(max_placements, activation ='softmax')(dense)

# Create the model
note_count_class_model = Model(inputs=[image_input, other_data_input], outputs=predictions) 
#model1.summary()

# Set optimizer and compile
#optimizer = keras.optimizers.Adam(lr=0.00001)
#optimizer = keras.optimizers.Adam(lr=0.0001)
optimizer = keras.optimizers.Adam(lr=0.001)
note_count_class_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy", keras_metrics.precision(), keras_metrics.recall()])
#model1.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['binary_accuracy'])

# Create generators
train_generator = CountDataGenerator(train_keys, song_df, num_mels, win_len, 
                                     chunk_radius, "Note", max_placements, batch_size)
validation_generator = CountDataGenerator(validation_keys, song_df, num_mels, win_len, 
                                          chunk_radius, "Note", max_placements, batch_size)
note_count_class_model.fit_generator(generator=train_generator, epochs=100, validation_data=validation_generator)
note_count_class_model.save('Experiments/Experiment1/note_count_class.h5')

In [ ]:
class TypeDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, key_difs, song_df, num_mels, win_len, chunk_radius, previous_steps, object_type="Note", batch_size=200):
        'Initialization'
        self.key_difs = key_difs
        self.song_df = song_df
        self.num_mels = num_mels
        self.win_len = win_len
        self.chunk_radius = chunk_radius
        self.previous_steps = previous_steps
        self.object_type = object_type
        self.batch_size = batch_size
        
        self.pitch_shift = 0
        self.pitch_index = 0
        self.pitch_shifts = [0, 4, -4, 8, -8, 12, -12, 16, -16]
        
        self.get_data_for_epoch()
        self.num_samples = self.specs.shape[0]

    def get_data_for_epoch(self):
        spec_list = []
        meta_list = []
        obj_list = []
        
        for key_dif in self.key_difs:
            key, dif = key_dif.split("-")
            
            meta_arr, spec, time_indices, obj_count_arr = get_song_count_train_data(self.song_df, key, self.num_mels, self.win_len, dif, 
                                                                self.pitch_shift, self.object_type)
            
            # Something else
            obj_df = get_pandas_song_map(song_df[(song_df.key == "570") & (song_df.difficulty == "Expert")].iloc[0])
            obj_df["input_num"] = round(obj_df["time"] * 1000 / win_len)
            
            obj_df = obj_df[obj_df.object_type == "Note"]
            obj_df = obj_df.drop(["time", "duration", "width", "object_type", "value"], axis=1)
            obj_df["type"] = obj_df["type"].astype(pd.api.types.CategoricalDtype(categories = [0, 1]))
            obj_df["line_index"] = obj_df["line_index"].astype(pd.api.types.CategoricalDtype(categories =[0, 1, 2, 3]))
            obj_df["line_layer"] = obj_df["line_layer"].astype(pd.api.types.CategoricalDtype(categories =[0, 1, 2]))
            obj_df["cut_direction"] = obj_df["cut_direction"].astype(pd.api.types.CategoricalDtype(categories =[0, 1, 2, 3, 4, 5, 6, 7, 8]))
            obj_df = pd.get_dummies(obj_df,prefix=["type", "index", "layer", "direction"])
            obj_df.sort_values(by=["input_num", "layer_0", "layer_1", "layer_2", "index_0", "index_1", "index_2", "index_3"])
            
            obj_arr = obj_df.to_numpy()
            obj_arr_index = 0
            
            max_index_dif = 500
            
            prev_indices = np.ones(self.previous_steps) * -max_index_dif
            prev_counts = np.zeros(self.previous_steps)
            
            for time_index in obj_arr[:, 0]:
                
                if time_index < meta_arr.shape[0]: # Sanity check for notes that have been placed after song end
                    time_difs = np.ones(self.previous_steps) * time_index - prev_indices
                    time_difs = np.clip(time_difs, 0, max_index_dif) / max_index_dif

                    # Stack the difference from the last time step, the counts at the last time steps, and the difficulty
                    # as the 
                    meta_info = np.hstack([time_difs, prev_counts, meta_arr[int(time_index), 0]]) 

                    spec_arr_new = chunk_spectrogram(spec, time_indices, time_index, 1, self.chunk_radius)
                    spec_list.append(spec_arr_new)
                    meta_list.append(meta_info) 
                    obj_list.append(obj_arr[obj_arr_index, 1:])
                    obj_arr_index += 1

                    if time_index == prev_indices[0]:
                        prev_counts[0] += 1
                    else:
                        for i in range(self.previous_steps - 1, 0, -1):
                            prev_indices[i] = prev_indices[i - 1]
                            prev_counts[i] = prev_counts[i - 1]

                        prev_indices[0] = time_index
                        prev_counts[0] = 1
        
        self.specs = reshape_spec_arr(np.vstack(spec_list))
        self.meta = np.stack(meta_list)
        self.objs = np.stack(obj_list)
        logging.debug("Shapes for epoch. Specs: {}, meta: {}, objs: {}".format(self.specs.shape, 
                                                                              self.meta.shape, self.objs.shape))
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.ceil(self.num_samples / self.batch_size)
    
    def on_epoch_end(self):
        'Update pitches after each epoc'
        
        self.pitch_index += 1
        
        if self.pitch_index >= len(self.pitch_shifts):
            self.pitch_index = 0
        
        self.pitch_shift = self.pitch_shifts[self.pitch_index]
        
        self.get_data_for_epoch()
    
    def __getitem__(self, index):
        'Generate one batch of data'
        #import pdb; pdb.set_trace()
        # Get key and difficulty for current         
        #x = [self.specs[index:index+batch_size], self.meta[index:index+batch_size]]
        x = self.meta[index:index+batch_size]
        y = self.objs[index:index+batch_size]
        
        return x, y

In [ ]:
previous_steps = 3
train_keys = train_keys_s
validation_keys = train_keys_s

In [ ]:
# Timing and difficulty information
other_data_input = Input(shape=(1 + previous_steps * 2, ))   

merged_model = Reshape((1, 1 + previous_steps * 2))(other_data_input)

# Add an extra dense layer between the data and the output
dense = LSTM(chunk_radius * 2 + 1, return_sequences=True, activation ='relu')(merged_model)
#dense = Dropout(.2)(dense)
dense = LSTM(chunk_radius * 2 + 1, return_sequences=True, activation ='relu')(dense)
dense = LSTM(chunk_radius * 2 + 1, return_sequences=True, activation ='relu')(dense)
dense = LSTM(chunk_radius * 2 + 1, return_sequences=True, activation ='relu')(dense)
dense = LSTM(chunk_radius * 2 + 1, activation ='relu')(dense)

# Predict on the output 
predictions = Dense(18, activation ='sigmoid')(dense)

# Create the model
note_type_class_model = Model(inputs=[other_data_input], outputs=predictions) 
note_type_class_model.summary()

# Set optimizer and compile
#optimizer = keras.optimizers.Adam(lr=0.00001)
#optimizer = keras.optimizers.Adam(lr=0.0001)
optimizer = keras.optimizers.Adam(lr=0.001)
note_type_class_model.compile(loss='binary_crossentropy', optimizer=optimizer, 
                              metrics=["accuracy", keras_metrics.precision(), keras_metrics.recall()])

# Create generators
train_generator = TypeDataGenerator(train_keys, song_df, num_mels, win_len, 
                                     chunk_radius, previous_steps, "Note", batch_size)
validation_generator = TypeDataGenerator(validation_keys, song_df, num_mels, win_len, 
                                          chunk_radius, previous_steps, "Note", batch_size)
note_type_class_model.fit_generator(generator=train_generator, epochs=50, validation_data=validation_generator)
note_type_class_model.save('Experiments/Experiment1/note_type_class.h5')

In [ ]:
def create_info_file_for_song(directory, song_name, bpm, sub_name="", author_name="", 
                              level_author_name="TheBeatBot", preview_start_time=10, preview_duration=10):

    with open("MetadataFiles/info.dat", 'r') as base_file:
        filedata = base_file.read()

    # Replace the target string
    filedata = filedata.replace('-song_name-', song_name)
    filedata = filedata.replace('-song_sub_name-', sub_name)
    filedata = filedata.replace('-song_author-', author_name)
    filedata = filedata.replace('-level_author-', level_author_name)
    filedata = filedata.replace('-preview_start-', str(preview_start_time))
    filedata = filedata.replace('-preview_duration-', str(preview_duration))
    filedata = filedata.replace('-bpm-', str(bpm))
    
    # convert into JSON:
    file_path = "{}/info.dat".format(directory)
    
    with open(file_path, 'w') as outfile:
        outfile.write(filedata)

In [ ]:
def run_place_model(spec, time_indices, meta_arr, chunk_radius, place_model, min_threshold):
    spec_arr = chunk_spectrogram(spec, time_indices, 0, spec.shape[1], chunk_radius)

    # Reshape the spectrogram to fit the expected 4 dimensions
    spec_arr = reshape_spec_arr(spec_arr)

    place_pred = place_model.predict([spec_arr, meta_arr])
    maximums = []
    max_vals = []

    #min_threshold = .5
    #import pdb; pdb.set_trace()

    index_len = len(place_pred)
    index_range = range(index_len - 1)

    for i in index_range:
        if (i > 0 and place_pred[i] > place_pred[i - 1] and i < index_len - 1 and place_pred[i] > place_pred[i + 1] 
            and place_pred[i] > min_threshold):
            maximums.append(i)
            max_vals.append(place_pred[i])
    
    return maximums, max_vals

def run_count_model(spec, time_indices, meta_arr, indices, chunk_radius, count_model):
    spec_list = []
    meta_list = []

    for index in indices:
        spec_arr_new = chunk_spectrogram(spec, time_indices, index, 1, chunk_radius)
        spec_list.append(spec_arr_new)
        meta_list.append(meta_arr[index, 0])

    spec_arr = reshape_spec_arr(np.vstack(spec_list))
    meta_arr = np.stack(meta_list)

    #import pdb; pdb.set_trace()
    return count_model.predict([spec_arr, meta_arr])

def run_type_model(spec, time_indices, meta_arr, indices, chunk_radius, previous_steps, max_index_dif, type_model):
    spec_list = []
    meta_list = []
    
    prev_indices = np.ones(previous_steps) * -max_index_dif
    prev_counts = np.zeros(previous_steps)

    for time_index in indices:
        if time_index < meta_arr.shape[0]: # Sanity check for notes that have been placed after song end
            time_difs = np.ones(previous_steps) * time_index - prev_indices
            time_difs = np.clip(time_difs, 0, max_index_dif) / max_index_dif

            # Stack the difference from the last time step, the counts at the last time steps, and the difficulty
            # as the 
            meta_info = np.hstack([time_difs, prev_counts, meta_arr[int(time_index), 0]]) 

            spec_arr_new = chunk_spectrogram(spec, time_indices, time_index, 1, chunk_radius)
            spec_list.append(spec_arr_new)
            meta_list.append(meta_info)

            if time_index == prev_indices[0]:
                prev_counts[0] += 1
            else:
                for i in range(previous_steps - 1, 0, -1):
                    prev_indices[i] = prev_indices[i - 1]
                    prev_counts[i] = prev_counts[i - 1]

                prev_indices[0] = time_index
                prev_counts[0] = 1
    
    spec_arr = reshape_spec_arr(np.vstack(spec_list))
    meta_arr = np.stack(meta_list)
    
    #return type_model.predict([spec_arr, meta_arr])
    return type_model.predict(meta_arr)

def index_to_beat(win_len, index, bpm):
    return (win_len / 1000) * index * bpm / 60

def create_map_file_for_song(directory, difficulty_name, notes):
    info_dict = {"_version": "2.0.0",
                 "_PBMChanges": [],
                 "_events": [],
                 "_notes": notes,
                 "_obstacles": [],
                 "_bookmarks": []}
    
    file_path = "{}/{}.dat".format(directory, difficulty_name)
    
    with open(file_path, 'w') as outfile:
        json.dump(info_dict, outfile)

def create_map_for_song(filename, num_mels, win_len, output_directory, 
                        previous_steps, max_index_dif, 
                        place_model, count_model, type_model,
                        song_name, sub_name="", author_name="", 
                        level_author_name="TheBeatBot", preview_start_time=10, preview_duration=10):
    
    y, sr = librosa.load(filename)
    bpm = librosa.beat.tempo(y, sr)[0]
    
    create_info_file_for_song(output_directory, song_name, bpm, sub_name="", author_name="", 
                              level_author_name="TheBeatBot", preview_start_time=10, preview_duration=10)
    
    spec, time_indices = get_spectrogram_from_file(filename, num_mels, win_len)
    input_len = len(time_indices)
    
    difficulties = ["Easy", "Normal", "Hard", "Expert", "ExpertPlus"]
    #difficulties = ["Expert"]
    
    for dif in difficulties:
        meta_arr = get_meta_arr(input_len, dif_val_from_string(dif), win_len)
        
        maximums, max_vals = run_place_model(spec, time_indices, meta_arr, chunk_radius, place_model, min_threshold=.5)
        
        count_pred = run_count_model(spec, time_indices, meta_arr, maximums, chunk_radius, count_model)
        
        type_indices = []

        cur_max_index = 0

        for num_indices in np.argmax(count_pred, axis=1):
            for i in range(num_indices):
                type_indices.append(maximums[cur_max_index])

            cur_max_index += 1
        
        type_pred = run_type_model(spec, time_indices, meta_arr, type_indices, chunk_radius, previous_steps, max_index_dif, type_model)
        
        # 0 - 1 = Type = value of 0 - 1
        # 2 - 5 = line index = value 0 - 3
        # 6 - 8 = line layer = value 0 - 2
        # 9 - 17 = cut direction = value 0 - 8
        pred_type = np.argmax(type_pred[:, 0:2], axis=1)
        pred_index = np.argmax(type_pred[:, 2:6], axis=1)
        pred_layer = np.argmax(type_pred[:, 6:9], axis=1)
        pred_direction = np.argmax(type_pred[:, 9:18], axis=1)
        
        notes = []
        #import pdb; pdb.set_trace()
        for i in range(len(type_indices)):
            note = {}
            note["_time"] = index_to_beat(win_len, type_indices[i], bpm)
            note["_lineIndex"] = int(pred_index[i])
            note["_lineLayer"] = int(pred_layer[i])
            note["_type"] = int(pred_type[i])
            note["_cutDirection"] = int(pred_direction[i])
            
            notes.append(note)
        
        
        create_map_file_for_song(output_directory, dif, notes)

In [ ]:
filename = "Experiments/Experiment1/Fallout 76.egg"
output_directory = "Experiments/Experiment1"
place_model = load_model('Experiments/Experiment1/note_place_class.h5', custom_objects={'binary_precision': keras_metrics.precision(), 'binary_recall': keras_metrics.recall()})

In [ ]:
count_model = load_model('Experiments/Experiment1/note_count_class.h5', custom_objects={'binary_precision': keras_metrics.precision(), 'binary_recall': keras_metrics.recall()})

In [ ]:
type_model = load_model('Experiments/Experiment1/note_type_class.h5', custom_objects={'binary_precision': keras_metrics.precision(), 'binary_recall': keras_metrics.recall()})

In [ ]:
create_map_for_song(filename, num_mels, win_len, output_directory, previous_steps, 500,
                     note_place_class_model, note_count_class_model, note_type_class_model, 
                     song_name="Test Song", sub_name="Test sub", author_name="Test Author Name", 
                     level_author_name="TestTheBeatBot", preview_start_time=10, preview_duration=10)